In [10]:
import dgl
import dgl.nn.pytorch as dglnn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
exc_feat_path = './data/train/feat/exc/'
dum_feat_path = './data/train/feat/dum/'
cos_feat_path = './data/train/feat/cos/'

exc_edge_path = './data/train/edge/exc/'
dum_edge_path = './data/train/edge/dum/'
cos_edge_path = './data/train/edge/cos/'


exc_edge_list = glob.glob(exc_edge_path + '*.txt')
dum_edge_list = glob.glob(dum_edge_path + '*.txt')
cos_edge_list = glob.glob(cos_edge_path + '*.txt')

exc_feat_list = glob.glob(exc_feat_path+'*.txt')
dum_feat_list = glob.glob(dum_feat_path+'*.txt')
cos_feat_list = glob.glob(cos_feat_path+'*.txt')

labels = pd.read_csv('./data/labels.txt', header=None, sep=' ').values

def create_graph(n):

    exc_edge = pd.read_csv(exc_edge_list[n], sep=',', header=None).to_numpy()  
    dum_edge = pd.read_csv(dum_edge_list[n], sep=',', header=None).to_numpy()
    cos_edge = pd.read_csv(cos_edge_list[n], sep=',', header=None).to_numpy()
    

    exc_src = exc_edge[:, 0]
    exc_dst = exc_edge[:, 1]

    dum_src = dum_edge[:, 0]
    dum_dst = dum_edge[:, 1]

    cos_src = cos_edge[:, 0]
    cos_dst = cos_edge[:, 1]
    # print(feature_data_list[n])
    # Read the entire feature data from the file
    
    exc_feat = pd.read_csv(exc_feat_list[n], sep='\t', header=None).to_numpy()
    dum_feat = pd.read_csv(dum_feat_list[n], sep='\t', header=None).to_numpy()
    cos_feat = pd.read_csv(cos_feat_list[n], sep='\t', header=None).to_numpy()

    # print(exc_feat.shape)

    # Define number of nodes for each type


    hetero_graph = dgl.heterograph({
        ('exc', 'inexc', 'exc'): (exc_src, exc_dst),
        ('exc', 'outexc', 'exc'): (exc_dst, exc_src),
        ('dum', 'indum', 'dum'): (dum_src, dum_dst), 
        ('dum', 'outdum', 'dum'): (dum_dst, dum_src),
        ('cos', 'incos', 'cos'): (cos_src, cos_dst), 
        ('cos', 'outcos', 'cos'): (cos_dst, cos_src), 
    })
    hetero_graph = hetero_graph.to(device)
    hetero_graph.nodes['exc'].data['feat'] = torch.tensor(exc_feat, dtype=torch.float32).to(device)
    hetero_graph.nodes['dum'].data['feat'] = torch.tensor(dum_feat, dtype=torch.float32).to(device)
    hetero_graph.nodes['cos'].data['feat'] = torch.tensor(cos_feat, dtype=torch.float32).to(device)

    label = labels[n]
    return hetero_graph, torch.LongTensor(label).to(device)

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

class HeteroClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes, rel_names):
        super().__init__()

        self.rgcn1 = RGCN(in_dim, hidden_dim, hidden_dim, rel_names)
        self.rgcn2 = RGCN(hidden_dim, hidden_dim, hidden_dim, rel_names)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g):
        h = g.ndata['feat']
        h = self.rgcn1(g, h)
        h = self.rgcn2(g, h)
        with g.local_scope():
            g.ndata['h'] = h
            hg = 0
            for ntype in g.ntypes:
                hg = hg + dgl.mean_nodes(g, 'h', ntype=ntype)
            return self.classify(hg)

# print(dataset[0])

def collate(samples):
    # graphs, labels = map(list, zip(*samples))
    graphs, labels = zip(*samples)
    batched_graph = dgl.batch(graphs)
    batched_labels = torch.tensor(labels)
    return batched_graph, batched_labels


dataset = [create_graph(n) for n in range(len(exc_feat_list))]


# print(dataset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:

train_dataset, val_dataset = train_test_split(dataset, test_size=0.1, random_state=137)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=256,
    collate_fn=collate,
    drop_last=False,
    shuffle=True)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=256,
    collate_fn=collate,
    drop_last=False)

etypes = dataset[0][0].etypes
# print(etypes)
model = HeteroClassifier(3, 256, 4, etypes).to(device)
opt = torch.optim.Adam(model.parameters())

loss_values = []
acc_values = []

for epoch in range(1000):
    # Training
    model.train()
    train_loss = 0
    for batched_graph, labels in train_dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        logits = model(batched_graph)
        loss = F.cross_entropy(logits, labels.squeeze(-1))
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_loss += loss.item()
    print(f'Epoch {epoch}, Training Loss: {train_loss / len(train_dataloader)}')

    # Validation
    model.eval()
    val_loss = 0
    val_acc = 0
    all_predictions = []
    all_labels = []
for epoch in range(1000):
    # Training
    model.train()
    train_loss = 0
    for batched_graph, labels in train_dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        logits = model(batched_graph)
        loss = F.cross_entropy(logits, labels.squeeze(-1))
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_loss += loss.item()
    print(f'Epoch {epoch}, Training Loss: {train_loss / len(train_dataloader)}')

    # Validation
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for batched_graph, labels in val_dataloader:
            batched_graph = batched_graph.to(device)
            labels = labels.to(device)
            logits = model(batched_graph)
            loss = F.cross_entropy(logits, labels.squeeze(-1))
            val_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(logits, 1)
            val_acc += (predicted == labels.squeeze(-1)).sum().item() / len(labels)

        print(f'Epoch {epoch}, Validation Loss: {val_loss / len(val_dataloader)}, Validation Accuracy: {val_acc / len(val_dataloader)}')



Epoch 0, Training Loss: 201.99470901489258
Epoch 1, Training Loss: 210.61272430419922
Epoch 2, Training Loss: 130.37279510498047
Epoch 3, Training Loss: 73.44104766845703
Epoch 4, Training Loss: 58.50636100769043
Epoch 5, Training Loss: 25.31825542449951
Epoch 6, Training Loss: 37.42384147644043
Epoch 7, Training Loss: 11.382310390472412
Epoch 8, Training Loss: 15.199971199035645
Epoch 9, Training Loss: 10.872858047485352
Epoch 10, Training Loss: 6.674008131027222
Epoch 11, Training Loss: 6.334394693374634
Epoch 12, Training Loss: 3.625358462333679
Epoch 13, Training Loss: 3.5836435556411743
Epoch 14, Training Loss: 3.073764681816101
Epoch 15, Training Loss: 2.6678911447525024
Epoch 16, Training Loss: 1.501495897769928
Epoch 17, Training Loss: 1.6743255853652954
Epoch 18, Training Loss: 1.8914850354194641
Epoch 19, Training Loss: 1.3522207736968994
Epoch 20, Training Loss: 0.9773918986320496
Epoch 21, Training Loss: 0.8705098032951355
Epoch 22, Training Loss: 0.869103729724884
Epoch 23